In [ ]:
import back_propogation.backprop as bp
from back_propogation.backprop import Node as node
import back_propogation.analytic_function as af
import numpy as np
import matplotlib.pyplot as pyplot

size = [10, 25, 50]
iteration = [2000, 2000, 2000]
X = np.load('assignment8_X.npy')
Y = np.load('assignment8_Y.npy')
alpha = 0.07

x = node('x')
w1 = node('w1')
h1 = node('h1')
w2 = node('w2')
h2 = node('h2')
w3 = node('w3')
h3 = node('h3')

x.toNode(h1)
w1.toNode(h1)
h1.toNode(h2)
w2.toNode(h2)
h2.toNode(h3)
w3.toNode(h3)

h1.analyze(af.funcMatVec(), [w1, x], True)
h2.analyze(af.funcMatVec(), [w2, h1], True)
h3.analyze(af.funcMatVec(), [w3, h2], True)

for index in range(3):
    m = size[index]
    ite = iteration[index]
    w1val = 0.01 * np.random.randn(m, 10)
    w2val = 0.01 * np.random.randn(m, m)
    w3val = 0.01 * np.random.randn(10, m)
    chain = []
    for i in range(ite):
        w1gradient = []
        w2gradient = []
        w3gradient = []
        loss = 0
        for j in range(len(X)):
            xval = X[j]
            yval = Y[j]
            bp.forward([x, w1, w2, w3], [xval, w1val, w2val, w3val])
            bp.backward(h3)
            w1gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w1.der[j]) for j in range(len(yval))]))
            w2gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w2.der[j]) for j in range(len(yval))]))
            w3gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w3.der[j]) for j in range(len(yval))]))
            loss += np.sum([(h3.value[j] - yval[j]) ** 2 for j in range(len(yval))])
            bp.reset(h3)
        w1val -= alpha * sum(w1gradient) / 25 * 2
        w2val -= alpha * sum(w2gradient) / 25 * 2
        w3val -= alpha * sum(w3gradient) / 25 * 2
        # if i % 25 == 0:
        #    print(loss/25)
        chain.append(loss/25)
    print(loss/25)
    print(np.dot(np.dot(w3val, w2val), w1val))
    fig, ax = pyplot.subplots()
    ax.plot(chain)

x = node('x')
w1 = node('w1')
h1 = node('h1')
v1 = node('v1')
w2 = node('w2')
h2 = node('h2')
v2 = node('v2')
w3 = node('w3')
h3 = node('h3')

x.toNode(h1)
w1.toNode(h1)
h1.toNode(v1)
v1.toNode(h2)
w2.toNode(h2)
h2.toNode(v2)
v2.toNode(h3)
w3.toNode(h3)

h1.analyze(af.funcMatVec(), [w1, x], True)
v1.analyze(af.funcSig(), [h1], True)
h2.analyze(af.funcMatVec(), [w2, v1], True)
v2.analyze(af.funcSig(), [h2], True)
h3.analyze(af.funcMatVec(), [w3, v2], True)

rates = [0.5, 0.25, 0.25]

for index in range(2):
    m = size[index]
    ite = iteration[index]
    alpha = rates[index]
    w1val = 0.01 * np.random.randn(m, 10)
    w2val = 0.01 * np.random.randn(m, m)
    w3val = 0.01 * np.random.randn(10, m)
    chain = []
    for i in range(ite):
        w1gradient = []
        w2gradient = []
        w3gradient = []
        loss = 0
        for j in range(len(X)):
            xval = X[j]
            yval = Y[j]
            bp.forward([x, w1, w2, w3], [xval, w1val, w2val, w3val])
            bp.backward(h3)
            w1gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w1.der[j]) for j in range(len(yval))]))
            w2gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w2.der[j]) for j in range(len(yval))]))
            w3gradient.append(sum([np.multiply((h3.value[j] - yval[j]), w3.der[j]) for j in range(len(yval))]))
            loss += np.sum([(h3.value[j] - yval[j]) ** 2 for j in range(len(yval))])
            bp.reset(h3)
        w1val -= alpha * sum(w1gradient) / 25 * 2
        w2val -= alpha * sum(w2gradient) / 25 * 2
        w3val -= alpha * sum(w3gradient) / 25 * 2
        # if i % 25 == 0:
        #    print(loss / 25)
        chain.append(loss/25)
    print(loss / 25)
    print(np.dot(np.dot(w3val, w2val), w1val))
    fig, ax = pyplot.subplots()
    ax.plot(chain)


w = np.random.randn(10, 10)
X = np.load('assignment9_X.npy').transpose()
Y = np.load('assignment9_Y.npy')
delta = 1
alpha = 0.001
chain = []

for ite in range(500):
    gradient = 0
    loss = 0
    for i in range(len(X)):
        xval = X[i]
        yval = Y[i]

        def difference(k):
            result = np.dot(w[k], xval) - np.dot(w[yval], xval)
            return result

        wsum = (-sum([0 if j == yval else (1 if difference(j) + delta > 0 else 0) for j in range(10)]) * xval).tolist()
        local = [[0] * 10 if j != yval else wsum for j in range(10)]
        gradient = np.add(gradient, local)
        loss += sum([0 if j == yval else (0 if difference(j) - delta < 0 else difference(j) - delta) for j in range(10)])
    w -= alpha * gradient
    chain.append(loss)
print(loss)
fig, ax = pyplot.subplots()
ax.plot(chain)



1.6610165176215312
[[ 0.05512672  0.01735212 -0.00987439 -0.10561055 -0.04534931  0.01999315
   0.15633367 -0.05546086  0.1016042   0.02049122]
 [ 0.18217112  0.00107958  0.02062149 -0.02449516 -0.02931825 -0.09316833
  -0.09080957 -0.01580618  0.05562524  0.00938393]
 [-0.0577974   0.07618713 -0.02672178 -0.0507522   0.00363797  0.08070436
  -0.0895389   0.10370837  0.05278518  0.01308797]
 [-0.06999531  0.07097899 -0.03058158 -0.05155645 -0.04114028  0.19102153
   0.04318534  0.01697603 -0.06291905  0.04416294]
 [-0.00428854  0.05248633  0.05201952 -0.18437303 -0.05753524  0.01586339
   0.06953181 -0.00446539  0.05181949  0.17184772]
 [ 0.13449136  0.03906269  0.02694166 -0.01217517 -0.0111815   0.01584393
  -0.09854758 -0.08466245 -0.11880619  0.04287184]
 [ 0.02725509 -0.00756639  0.13383175 -0.00364453 -0.0766255   0.05851491
  -0.03350224 -0.02561942 -0.060854    0.0392354 ]
 [-0.08918413 -0.12040379  0.08553892  0.11698906  0.07155652 -0.21068873
   0.03177916  0.00563693 -0.037